In [1]:
from bastionlab import Connection, Identity
import polars as pl

In [2]:
data_owner = Identity.create("data_owner")
client = Connection("localhost", identity=data_owner).client

In [3]:
target_names = ["setosa", "versicolor", "virginica"]
df = pl.read_csv(
    "iris.csv",
    has_header=False,
    new_columns=[
        "sepal length (cm)",
        "sepal width (cm)",
        "petal length (cm)",
        "petal width (cm)",
        "target",
    ],
)


df = df.with_column(
    pl.col("target")
    .map(lambda a: a.apply(lambda b: True if target_names[b] == "virginica" else False))
    .alias("target_class")
)

target_class_df = pl.DataFrame({"classes": target_names})
target_class_df = target_class_df.with_row_count("id")

In [4]:
rdf = client.polars.send_df(df)
classes_rdf = client.polars.send_df(target_class_df)

train_rdf = rdf.select(pl.col(["petal length (cm)", "petal width (cm)"])).collect()
test_rdf = rdf.select(pl.col("target_class")).collect()
print(train_rdf, test_rdf)

FetchableLazyFrame(identifier=6c367977-8bc2-42e2-936e-d38fbd8c2a6a) FetchableLazyFrame(identifier=6c795f46-6f09-42e5-b905-827c9acab73a)


In [5]:
from bastionlab.linfa.trainers import LogisticRegression

trainer = client.linfa.train(train_rdf, test_rdf, LogisticRegression())

In [6]:
res = client.linfa.predict(trainer, [5.5, 4.2])
print(res)

shape: (1, 1)
┌────────────┐
│ prediction │
│ ---        │
│ u64        │
╞════════════╡
│ 1          │
└────────────┘


In [9]:
res = client.linfa.predict_proba(trainer, [5.5, 4.2])
print(res)

shape: (1, 1)
┌────────────┐
│ prediction │
│ ---        │
│ f64        │
╞════════════╡
│ 1.0        │
└────────────┘


In [ ]:
res = client.linfa.cross_validate(trainer)